In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import urllib.request
import matplotlib.pyplot as plt
import tensorflow as tf

import cv2
from sklearn.model_selection import train_test_split

from keras.datasets import cifar10
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import Sequential, load_model, Model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [3]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=0)

print('Train Images Shape:      ', X_train.shape)
print('Train Labels Shape:      ', y_train.shape)

print('\nValidation Images Shape: ', X_valid.shape)
print('Validation Labels Shape: ', y_valid.shape)

print('\nTest Images Shape:       ', X_test.shape)
print('Test Labels Shape:       ', y_test.shape)

X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')
X_test = X_test.astype('float32')

mean = np.mean(X_train, axis=(0, 1, 2, 3))
std = np.std(X_train, axis=(0, 1, 2, 3))

X_train = (X_train - mean) / (std + 1e-7)
X_valid = (X_valid - mean) / (std + 1e-7)
X_test = (X_test - mean) / (std + 1e-7)

y_train = to_categorical(y_train, 10)
y_valid = to_categorical(y_valid, 10)
y_test = to_categorical(y_test, 10)

data_generator = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    shear_range=0.1,
    zoom_range=0.1
)

data_generator.fit(X_train)

Train Images Shape:       (45000, 32, 32, 3)
Train Labels Shape:       (45000, 1)

Validation Images Shape:  (5000, 32, 32, 3)
Validation Labels Shape:  (5000, 1)

Test Images Shape:        (10000, 32, 32, 3)
Test Labels Shape:        (10000, 1)


In [4]:
def create_cnn_model():
    model = Sequential()
    weight_decay = 0.0001
    
    model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', kernel_regularizer=l2(weight_decay), input_shape=X_train.shape[1:], name='conv1_1'))
    model.add(BatchNormalization(name='bn1_1'))
    model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', kernel_regularizer=l2(weight_decay), name='conv1_2'))
    model.add(BatchNormalization(name='bn1_2'))
    model.add(MaxPooling2D(pool_size=(2, 2), name='pool1'))
    model.add(Dropout(rate=0.2, name='drop1'))
    
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', kernel_regularizer=l2(weight_decay), name='conv2_1'))   
    model.add(BatchNormalization(name='bn2_1'))
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', kernel_regularizer=l2(weight_decay), name='conv2_2'))
    model.add(BatchNormalization(name='bn2_2'))
    model.add(MaxPooling2D(pool_size=(2, 2), name='pool2'))
    model.add(Dropout(rate=0.3, name='drop2'))
    
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', kernel_regularizer=l2(weight_decay), name='conv3_1')) 
    model.add(BatchNormalization(name='bn3_1'))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', kernel_regularizer=l2(weight_decay), name='conv3_2'))
    model.add(BatchNormalization(name='bn3_2'))
    model.add(MaxPooling2D(pool_size=(2, 2), name='pool3'))
    model.add(Dropout(rate=0.4, name='drop3'))
    
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu', kernel_regularizer=l2(weight_decay), name='conv4_1')) 
    model.add(BatchNormalization(name='bn4_1'))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu', kernel_regularizer=l2(weight_decay), name='conv4_2'))
    model.add(BatchNormalization(name='bn4_2'))
    model.add(MaxPooling2D(pool_size=(2, 2), name='pool4'))
    model.add(Dropout(rate=0.5, name='drop4'))
    
    model.add(Flatten(name='flatten'))
    
    model.add(Dense(10, activation='softmax', name='predictions'))
    
    return model

model = create_cnn_model()
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1_1 (Conv2D)                │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn1_1 (BatchNormalization)      │ (None, 32, 32, 32)     │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_2 (Conv2D)                │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn1_2 (BatchNormalization)      │ (None, 32, 32, 32)     │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1 (MaxPooling2D)            │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop1 (Dropout)                 │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2_1 (Conv2D)                │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn2_1 (BatchNormalization)      │ (None, 16, 16, 64)     │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2_2 (Conv2D)                │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn2_2 (BatchNormalization)      │ (None, 16, 16, 64)     │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2 (MaxPooling2D)            │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop2 (Dropout)                 │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3_1 (Conv2D)                │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn3_1 (BatchNormalization)      │ (None, 8, 8, 128)      │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3_2 (Conv2D)                │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn3_2 (BatchNormalization)      │ (None, 8, 8, 128)      │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool3 (MaxPooling2D)            │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop3 (Dropout)                 │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv4_1 (Conv2D)                │ (None, 4, 4, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn4_1 (BatchNormalization)      │ (None, 4, 4, 256)      │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv4_2 (Conv2D)                │ (None, 4, 4, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn4_2 (BatchNormalization)      │ (None, 4, 4, 256)      │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool4 (MaxPooling2D)            │ (None, 2, 2, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop4 (Dropout)                 │ (None, 2, 2, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             

 Total params: 1,186,346 (4.53 MB)

 Trainable params: 1,184,426 (4.52 MB)

 Non-trainable params: 1,920 (7.50 KB)

In [ ]:
batch_size = 64
epochs = 300

optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=40, restore_best_weights=True, verbose=1)

try:
    model = load_model('cifar10_cnn_model.h5')
    print("Modelo pré-treinado carregado com sucesso.")
except:
    print("Modelo pré-treinado não encontrado. Iniciando o treinamento...")
    model.fit(
        data_generator.flow(X_train, y_train, batch_size=batch_size),
        epochs=epochs,
        validation_data=(X_valid, y_valid),
        callbacks=[reduce_lr, early_stopping], 
        verbose=2
    )
    model.save('cifar10_cnn_model.h5')
    print("Modelo treinado e salvo.")

Modelo pré-treinado não encontrado. Iniciando o treinamento...
Epoch 1/300
704/704 - 44s - 62ms/step - accuracy: 0.3481 - loss: 2.2978 - val_accuracy: 0.4488 - val_loss: 1.8401 - learning_rate: 5.0000e-04
Epoch 2/300
704/704 - 40s - 57ms/step - accuracy: 0.4684 - loss: 1.7058 - val_accuracy: 0.5316 - val_loss: 1.4169 - learning_rate: 5.0000e-04
Epoch 3/300
704/704 - 40s - 56ms/step - accuracy: 0.5330 - loss: 1.4845 - val_accuracy: 0.5700 - val_loss: 1.3281 - learning_rate: 5.0000e-04
Epoch 4/300
704/704 - 40s - 57ms/step - accuracy: 0.5830 - loss: 1.3211 - val_accuracy: 0.6568 - val_loss: 1.0430 - learning_rate: 5.0000e-04
Epoch 5/300
704/704 - 41s - 58ms/step - accuracy: 0.6189 - loss: 1.2089 - val_accuracy: 0.6772 - val_loss: 1.0235 - learning_rate: 5.0000e-04
Epoch 6/300
704/704 - 40s - 57ms/step - accuracy: 0.6523 - loss: 1.1095 - val_accuracy: 0.6704 - val_loss: 1.0661 - learning_rate: 5.0000e-04
Epoch 7/300


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)

print('\nTest Accuracy:', test_acc)
print('Test Loss:    ', test_loss)

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

url = "https://raw.githubusercontent.com/FarzadNekouee/Keras-CIFAR10-CNN-Model/master/truck_sample.png"
resp = urllib.request.urlopen(url)
image = np.asarray(bytearray(resp.read()), dtype="uint8")
image = cv2.imdecode(image, cv2.IMREAD_UNCHANGED)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

original_image = image.copy()

image = cv2.resize(image, (32,32))
image = image.astype('float32')
image = (image-mean)/(std+1e-7)

input_image = image.reshape((1, 32, 32, 3))

prediction = model.predict(input_image)
predicted_class = prediction.argmax()
print('Predicted class: ', class_names[predicted_class])

plt.imshow(original_image)
plt.title(f'Imagem Original - Predição: {class_names[predicted_class]}')
plt.xticks([])
plt.yticks([])
plt.grid(False)
plt.show()

In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # 1. Criar um modelo que mapeia a entrada para a saída da última camada convolucional
    # e para a saída final (logits)
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # 2. Calcular os gradientes da classe prevista em relação aos mapas de características
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # 3. Este é o gradiente do neurônio de saída (classe prevista) em relação
    # aos mapas de características da última camada convolucional
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # 4. Este é o vetor onde cada entrada é a média da intensidade dos gradientes
    # em um mapa de características específico
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # 5. Multiplicar cada canal nos mapas de características pela 'importância' de cada canal
    # em relação à classe alvo
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # 6. Para visualização, normalizar o mapa de calor entre 0 e 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# Função para exibir o mapa de calor sobre a imagem original
def display_gradcam(img, heatmap, alpha=0.4):
    # Redimensionar o mapa de calor para o tamanho da imagem original
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

    # Converter o mapa de calor para RGB
    heatmap = np.uint8(255 * heatmap)
    
    # Aplicar um mapa de cores (por exemplo, JET)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    # Converter a imagem original para float
    superimposed_img = np.uint8(img)
    
    # Sobrepor o mapa de calor na imagem original
    superimposed_img = cv2.addWeighted(superimposed_img, 1 - alpha, heatmap, alpha, 0)

    # Retornar a imagem sobreposta e o mapa de calor (para plotagem separada)
    return superimposed_img, heatmap

# Lista de nomes de camadas convolucionais para análise (excluindo BN e Dropout)
conv_layer_names = [
    'conv1_2', 
    'conv2_2', 
    'conv3_2', 
    'conv4_2'
]

# Plotar os mapas de saliência para cada camada
num_layers = len(conv_layer_names)
fig, axes = plt.subplots(2, num_layers, figsize=(4 * num_layers, 8))
fig.suptitle(f'Mapas de Saliência Grad-CAM para a Classe: {class_names[predicted_class]}', fontsize=16)

for i, layer_name in enumerate(conv_layer_names):
    # 1. Gerar o mapa de calor
    heatmap = make_gradcam_heatmap(
        input_image, model, layer_name, predicted_class
    )
    
    # 2. Exibir o mapa de calor puro
    axes[0, i].matshow(heatmap)
    axes[0, i].set_title(f'Mapa de Calor - {layer_name}', fontsize=10)
    axes[0, i].axis('off')
    
    # 3. Sobrepor na imagem original
    superimposed_img, _ = display_gradcam(original_image, heatmap)
    
    # 4. Exibir a imagem sobreposta
    axes[1, i].imshow(superimposed_img)
    axes[1, i].set_title(f'Sobreposição - {layer_name}', fontsize=10)
    axes[1, i].axis('off')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()